In [357]:
#imports
import numpy as np
import math
from PIL import Image, ImageOps
import io
import cv2
from scipy import stats
import copy
import time
import random
import os
from star_data_generator import *

In [234]:
def calculate_distance(star_i, star_j):
    return np.arccos(np.sin(star_j[1])*np.sin(star_i[1]) + np.cos(star_j[1])*np.cos(star_i[1])*np.cos(star_j[0]-star_i[0]))


In [259]:
#cartesian coordinates
def dist1(star_i, star_j):
    return np.arccos((np.dot(star_i, star_j)))

In [236]:
def to_radians(degrees):
    for i in range(len(degrees)):
        if not isinstance(degrees[i], float):
            degrees[i] = to_radians(degrees[i])
        else:
            degrees[i] = math.radians(degrees[i])
    return degrees

In [485]:
# Create Feature Set
def create_feature_set(catalog, FOV):
    feature_set = [] # [(id(0), id(1)), theta < FOV]
    for i in range(len(catalog)):
        for j in range(i+1, len(catalog)):
            theta = dist1(catalog[i], catalog[j])
            #theta = np.abs(calculate_distance(catalog[i], catalog[j]))
            if theta <= FOV:
                feature_set.append((i, j, theta))
    return feature_set

In [238]:
#check if star is in poles list
def is_same(star, pole):
    print(star, pole)
    return star[0] == pole[0] and star[1] == pole[1]

In [486]:
#Create neighbour set of first pole using angular distances
def create_distance_set(stars, pole):
    return [dist1(star, pole) for star in stars]
    #return [np.abs(calculate_distance(star, pole)) for star in stars]

In [497]:
def create_pair_list_long(feature_set, distances, eps_pi):
    pairs = []
    for distance in distances:
        for i in range(len(feature_set)):
            if np.abs(feature_set[i][2] - distance) <= eps_pi:
                pairs.append((feature_set[i][0], feature_set[i][1]))
    return pairs

In [511]:
#pole is one that occurs most in list

def find_pole_in_pairs(pair_list):
    counts = {}
    for pair in pair_list:
        for star in pair:
            if star in counts:
                counts[star] += 1
            else:
                counts[star] = 1
    max_count = max(list(counts.values()))
    position = list(counts.values()).index(max_count)
    return list(counts.keys())[position]

"""import statistics
from statistics import mode
def find_pole_in_pairs(pairs):
    pairs = [ID for pair in pairs for ID in pair]
    return mode(pairs)

print(find_pole_in_pairs([[1, 3], [1, 3]]))"""

'import statistics\nfrom statistics import mode\ndef find_pole_in_pairs(pairs):\n    pairs = [ID for pair in pairs for ID in pair]\n    return mode(pairs)\n\nprint(find_pole_in_pairs([[1, 3], [1, 3]]))'

In [244]:
# neighbours are all stars paired with pole

def create_neighbour_set(pairs, pole):
    pole_neighbours = []
    for pair in pairs:
        if pair[0] == pole:
            pole_neighbours.append(pair[1])
        elif pair[1] == pole:
            pole_neighbours.append(pair[0])
    return np.unique(np.array(pole_neighbours)).tolist()

def create_neighbour_id_set(pairs, pole):
    return [ID for pair in pairs for ID in pair if pole in pair and ID != pole]

In [245]:
def intersection(set_1, set_2):
    intersection = [star for star in set_1 if star in set_2]
    return intersection

In [487]:
def initialise(stars, ids, num_poles):
    distance = []
    for i in range(len(stars)):
        distance.append((stars[i], dist1(stars[i], [0.5, 0.5, 0.5]), ids[i]))
        #distance.append([calculate_distance(stars[i], [512, 512]), ids[i]])
    distance = np.array(list(distance), dtype=[('position', object), ('distance', object), ('id', object)])
    distance = np.sort(distance, order='distance')
    return distance[:num_poles]

In [520]:
#returns intersection list if u and w true, else returns 0
def verification_phase(pole_i, neighbours_i, pole_j, neighbours_j):
    th = 3
    u = (pole_i in neighbours_j) and (pole_j in neighbours_i)
    if u:
        V = intersection(neighbours_i, neighbours_j)
        w = len(V) >= th
        if w:
            V.append(pole_j)
            return V
    return False

In [522]:
def confirmation_phase(verified, confirmed, features):
    print(verified)
    if len(verified) == 0:
        return confirmed
    v = verified[0]
    verified = verified[1:]
    verified_pos = [RA[ids.index(v)], DE[ids.index(v)]]
    confirmed_pos = [RA[ids.index(confirmed[-1])], DE[ids.index(confirmed[-1])]]
    displacement = dist1(verified_pos, confirmed_pos)
    #displacement = np.abs(calculate_distance(verified_pos, confirmed_pos))
    confirmed_features = [feature for feature in features if confirmed[-1] in feature]
    pairs = create_pair_list_long(confirmed_features, [displacement], eps_pi)
    pairs = [pair for pair in pairs if v in pair]
    if len(pairs) == 0:
        print(v, "not found")
        return confirmation_phase(verified, confirmed, features)
    else:
        confirmed.append(v)
        return confirmation_phase(verified, confirmed, features)

In [506]:
#returns pole and neighbour ids
def acceptance_phase(stars, features, candidate_pole, eps_pi):
    print("stars", len(stars))
    distances = create_distance_set(stars, candidate_pole)
    print("distance", len(distances))
    pairs = create_pair_list_long(features, distances, eps_pi)
    print("pairs", len(pairs))
    pole = find_pole_in_pairs(pairs)
    print("pole", pole)
    neighbours = create_neighbour_set(pairs, pole)
    
    return pole, neighbours, pairs

def acceptance_phase_1(stars, features, candidate_pole, eps_pi):
    print(len(stars), "stars")
    neighbours = [star for star in stars if not is_same(star, candidate_pole)]
    print(len(neighbours), "neighbours")
    #stars = [dist1(neighbour, candidate_pole) for neighbour in neighbours]
    stars = [np.abs(calculate_distance(neighbour, candidate_pole)) for neighbour in neighbours]
    print(len(stars), "distancess")

In [518]:
def recursive_MPA(stars, pole_i, neighbours_i, candidate_poles, N, features):
    u, w = False, False
    pole_j, neighbours_j, pairs = acceptance_phase(stars, features, candidate_poles['position'][N], eps_pi)
    print("ground truth", ids)
    print("ground truth id", candidate_poles['id'][N])
    print(pole_j, neighbours_j)
    verified = verification_phase(pole_i, neighbours_i, pole_j, neighbours_j)
    print(verified)
    if verified != False:
        return confirmation_phase(verified, [pole_i], features)
    if N == 0:
        return 0
    return recursive_MPA(stars, pole_i, neighbours_i, candidate_poles, N-1, features)


In [523]:
def image_to_ra_de(coords):
    x, y, z = coords[0], coords[1], coords[2]
    ra = np.arctan2(y, x)
    de = np.arcsin(z)
    return np.array([ra, de])

def get_data_from_file(file):
    catalog = []
    for line in file:
        line = list(map(float, line.split()))
        line = line/np.linalg.norm(line)
        catalog.append(line)
        #catalog.append(image_to_ra_de(line))
    return np.array(catalog)

FOV = np.deg2rad(12)
eps_pi = 0.0001

#file = open("HIP_catalog_vectors_with_mag_less_than_6", "r")
#catalog = get_data_from_file(file)
#features = create_feature_set(catalog, FOV)

data = (os.listdir('data'))
for i in range(1):
    stars = []
    #if data[i][0] == 's':
    #number = data[i][6:]
    number = '0'
    file = open("data/stars_" + number)
    stars = get_data_from_file(file)
    file = open("data/id_" + number)
    ids = []
    for line in file:
        ids.append(int(line))
    candidate_poles = initialise(stars, ids, int(len(ids)/2))
    N = len(candidate_poles)-1
    pole_i, neighbours_i, pairs = acceptance_phase(stars, 
                                        features, 
                                        candidate_poles['position'][N], 
                                        eps_pi)
    
    print("ground truth", ids)
    print("ground truth id", candidate_poles['id'][N])
    print("recursive")
    chain_set = recursive_MPA(stars,
                        pole_i, #first pole
                        neighbours_i, #first pole's neighbours
                        candidate_poles, #rest of poles
                        N-1, #j-th index
                        features) #[id0, id1, theta]
    print(pole_i, neighbours_i)


stars 10
distance 10
pairs 1497
pole 2156
ground truth [2199, 2224, 2156, 2219, 2277, 2067, 2145, 2158, 2121, 2170]
ground truth id 2156
recursive
stars 10
distance 10
pairs 1278
pole 2465
ground truth [2199, 2224, 2156, 2219, 2277, 2067, 2145, 2158, 2121, 2170]
ground truth id 2199
2465 [2276, 2356, 2363, 2377, 2439, 2617]
False
stars 10
distance 10
pairs 1799
pole 2224
ground truth [2199, 2224, 2156, 2219, 2277, 2067, 2145, 2158, 2121, 2170]
ground truth id 2224
2224 [2067, 2145, 2156, 2158, 2186, 2199]
[2067, 2145, 2199, 2224]
[2067, 2145, 2199, 2224]


NameError: name 'RA' is not defined

In [492]:
file = open("HIP_catalog_vectors_with_mag_less_than_6", "r")
catalog = get_data_from_file(file)
print(len(catalog))
print(ids[0], candidate_poles['id'][N])
print(catalog[ids[0]], catalog[candidate_poles['id'][N]])
print(dist1(catalog[ids[0]], catalog[candidate_poles['id'][N]]))
print(np.deg2rad(12))

5041
2199 2156
[-0.5833317   0.3767015   0.71959718] [-0.49564678  0.36400581  0.78856454]
0.11233693736814004
0.20943951023931956
49
49


In [437]:
print("image position", stars[1], stars[0])
print("catalog position", catalog[ids[1]], catalog[ids[0]])
print("image distance", dist1(stars[1], stars[0]))
print("catalog distance", dist1(catalog[ids[1]], catalog[ids[0]]))

image position [-0.54462526  1.48899887] [1.4032962  1.55046836]
catalog position [2.60789602 0.7165571 ] [2.56819271 0.80322204]
image distance 1.0862647463745432
catalog distance 0.03497242173724754


In [446]:
def image_to_ra_de1(coords):
    x, y, z = coords[0], coords[1], coords[2]
    ra = np.arctan2(y, x)
    de = np.arcsin(z)
    return np.array([ra, de])

def get_data_from_file(file):
    catalog = []
    for line in file:
        line = list(map(float, line.split()))
        line = line/np.linalg.norm(line)
        catalog.append(line)
        #catalog.append(image_to_ra_de(line))
    return np.array(catalog)

file = open("HIP_catalog_vectors_with_mag_less_than_6", "r")
catalog = get_data_from_file(file)
file = open("data/id_0")
a = []
for line in file:
    a.append(catalog[int(line)])
print(a[0])
print(dist1(a[0], a[1]))


file = open("data/stars_0")
b = get_data_from_file(file)

print(dist1(b[0], b[1]))
#print(image_to_ra_de1(a))

[-0.5833317   0.3767015   0.71959718]
0.09130830821251108
0.091257247434918


graphs x axis is pixel noise, y axis is id rate
same for x axis is mag noise
same for x axis is false stars percentage against true stars
each graph for each FOV
return file with numbers
implement id rate metric
implement id = -1 for unidentified stars
1. id rate = frac{num matched}{total number of stars}
2. measure false recognition rate. False positives/total N
3. No result. number of times back out of verification/total times run

In [381]:
c = StarCatalog()
print(c.catalog.RAdeg.values.tolist()[0])
print(c.catalog.DEdeg.values.tolist()[0])
file = open("HIP_catalog_vectors_with_mag_less_than_6", "r")
catalog = get_data_from_file(file)
print(catalog[0])
ids = c.catalog.HIP.values.tolist()
list(ids.index(4916))

0.26915958
-48.80985919
[ 0.00582595 -0.87855335]


ValueError: 4916 is not in list

In [22]:
#MPA Algorithm
start = time.time()

FOV = np.deg2rad(12)
catalog = StarCatalog()
ids = catalog.catalog.HIP.values.tolist()
RA = np.deg2rad(catalog.catalog.RAdeg.tolist())
DE = np.deg2rad(catalog.catalog.DEdeg.tolist())
eps_pi = 0.0001

candidate_poles, pole_ids = initialise()
#features = create_feature_set()
N = len(candidate_poles)-1
pole_i, neighbours_i = acceptance_phase(features, 
                                        candidate_poles[N], 
                                        eps_pi)
chain_set = recursive_MPA(pole_i, #first pole
    neighbours_i, #first pole's neighbours
    candidate_poles, #rest of poles
    N-1, #j-th index
    features) #[id0, id1, theta]
print((chain_set))
print((scene.ids))

end = time.time()
total_time = end - start
print("\n"+ str(total_time))

11784
[8814, 9001, 9021, 9640, 9977, 10064, 10644, 11090, 11840, 12086, 12181, 12623, 11784]
[12072, 8814, 9001, 9021, 9640, 9977, 10064, 10644, 11090, 11840, 12086, 12181, 12623, 11784]
[ 8814  9001  9021  9640  9977 10064 10644 11090 11784 11840 12072 12086
 12181 12623]

4.917223930358887
